In [1]:
import torch
import pandas as pd
df = pd.read_csv("all_data.csv")

In [2]:
import numpy as np
import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

In [3]:
import torch.utils.data as data_utils

In [4]:
train = pd.read_csv("train.csv")

In [5]:
test = pd.read_csv("test.csv")

In [6]:
train_X = train.loc[:, train.columns != 'totals.transactionRevenue']
train_y = train['totals.transactionRevenue'].values

In [7]:
test_X = test.loc[:, test.columns != 'totals.transactionRevenue']
test_y = test['totals.transactionRevenue'].values

In [8]:
train_data = np.array(train_X)#np.ndarray()
train_X_list=train_data.tolist()#list

In [9]:
train_torch = torch.FloatTensor(train_X_list)

In [10]:
print(type(train_torch))

<class 'torch.Tensor'>


In [11]:
target = torch.FloatTensor(train_y)

In [12]:
loss_list = []

In [13]:
print(type(target))

<class 'torch.Tensor'>


In [14]:
training_data = train_torch.unsqueeze(0)
training_data = training_data.permute(1,0,2)

In [15]:
target = target.unsqueeze(0)
target = target.permute(1,0)

In [16]:
train = data_utils.TensorDataset(training_data, target)

In [17]:
import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import torch.utils.data as data_utils

In [18]:
class CNN_net(nn.Module):
    def __init__(self,str_cols, num_cols):
        
        super(CNN_net, self).__init__()
        
        self.fc1 = nn.Sequential(

            nn.Conv1d(1, 16, kernel_size=3),
            nn.BatchNorm1d(16),
            nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
        self.fc2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3),
            nn.BatchNorm1d(32),
#             nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
#         self.fc3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear((len(str_cols) + len(num_cols) - 4 ) * 32,1)
        self.fc4 = nn.Linear(288,1)
#         self.fc5 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        out = x.reshape(x.size(0), -1)
#         print(out.size())
#         out = self.fc3(out)
        f = self.fc4(out)
#         f = self.fc5(f)
        return f


In [19]:
str_cols = ["channelGrouping", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "customDimensions.value"]

#num_cols = ["year", "month","weekday","hour","minute","second","totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits'] 
num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits'] 

In [20]:
net = CNN_net(str_cols, num_cols)
# net = net.cuda()
# print(net )

In [21]:
#zero the gradient buffers
net.zero_grad()
loss_list = []

epoch = 600

In [22]:
#create the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=51e-5)
criterion = nn.MSELoss()

In [ ]:
for i in range(epoch):
#     if i%100 == 0:
    print(i)
#     torch.cuda.empty_cache()
    running_loss = 0
    train_loader = data_utils.DataLoader(train, batch_size=256, shuffle=True)
    for trainx,labelx in train_loader:
        #print(trainx.size())
        optimizer.zero_grad()
        loss = criterion(net(trainx), labelx)
        loss_list.append(loss)
        loss.backward()
        optimizer.step() 
        running_loss += float(loss.item())
        loss_list.append(running_loss)
    print(running_loss)

0
